### Init

In [ ]:
import kutils
from kutils import model_helper as mh
from kutils import applications as apps
from kutils import tensor_ops as ops
from kutils import generic as gen

# ops.GPUMemoryCap(fraction=0.1)

In [ ]:
import pandas as pd
import numpy as np
import os
root_path = '/mnt/home/research/ava-mlsp/'
dataset = root_path + 'metadata/AVA_data_official_test.csv';
ids = pd.read_csv(dataset)

### Train on narrow MLSP features

In [ ]:
features_file = root_path + 'features/irnv2_mlsp_narrow_orig/i1[orig]_lfinal_o1[16928]_r1.h5'

fc1_size = 2048
image_size = '[orig]'
input_size = 16928
model_name = features_file.split('/')[-2]

loss = 'MSE'
bn = 2
fc_sizes = [fc1_size, fc1_size/2, fc1_size/8,  1]
dropout_rates = [0.25, 0.25, 0.5, 0]

monitor_metric = 'val_loss'; monitor_mode = 'min'
metrics = ["MAE", ops.plcc_tf]
outputs = 'MOS'

# MODEL DEF
from keras.layers import Input
from keras.models import Model
import keras

input_feats = Input(shape=(input_size,), dtype='float32')

pred = apps.fc_layers(input_feats, 
                 name          = 'head',
                 fc_sizes      = fc_sizes, 
                 dropout_rates = dropout_rates,
                 batch_norm    = bn)

model = Model(inputs=input_feats, outputs=pred)

gen_params = dict(batch_size    = 128,
                  data_path     = features_file,
                  input_shape   = (input_size,),
                  inputs        = 'image_name',
                  outputs       = outputs,
                  random_group  = False,
                  fixed_batches = True)

helper = mh.ModelHelper(model, model_name, ids, 
                     max_queue_size = 128,
                     loss           = loss,
                     metrics        = metrics,
                     monitor_metric = monitor_metric, 
                     monitor_mode   = monitor_mode,
                     early_stop_patience = 5,
                     multiproc      = True, workers = 3,
                     logs_root      = root_path + 'logs',
                     models_root    = root_path + 'models',
                     gen_params     = gen_params)

helper.model_name.update(fc1='[%d]'%fc1_size, 
                         im = image_size,
                         bn = bn,
                         do = str(dropout_rates).replace(' ',''),
                         mon = '[%s]' % monitor_metric,
                         ds = '[%s]' % os.path.split(dataset)[1])

print helper.model_name()

In [ ]:
# validate all at once
valid_set = ids[ids.set == 'validation']
valid_gen = helper.make_generator(valid_set,
                                  batch_size    = len(valid_set),
                                  shuffle       = False,
                                  deterministic = False)

for lr in [1e-3,1e-4,1e-5]:
    helper.load_model()
    helper.train(lr=lr, epochs=20, valid_gen=valid_gen)

### Test the model without augmentation

In [ ]:
model_name = root_path + 'models/irnv2_mlsp_narrow_orig/model'
# uncomment the following line to use the trained model (default)
# model_name = ''
if helper.load_model(model_name=model_name):
    y_test, y_pred, SRCC_test, PLCC_test, ACC_test =\
            apps.test_rating_model(helper, accuracy_thresh=5)